In [1]:
import os

In [2]:
%pwd

'g:\\Machine_Learning_Projects\\iNeuron internship\\Flight-Fare-Prediction-End-to-End-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'g:\\Machine_Learning_Projects\\iNeuron internship\\Flight-Fare-Prediction-End-to-End-ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [9]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
# New Line
from sklearn.ensemble import ExtraTreesRegressor

In [63]:
class DataTransformation:

    # New Function Added
    # https://github.com/yash1314/Flight-Price-Prediction/blob/main/src/utils.py
    def convert_to_minutes(self, duration):
        try:
            hours, minute = 0, 0
            for i in duration.split():
                if 'h' in i:
                    hours = int(i[:-1])
                elif 'm' in i:
                    minute = int(i[:-1])
            return hours * 60 + minute
        except :
            return None 

    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up

    # New Code Added Start
    def initiate_data_transformation(self):
        ## reading the data
        # df = pd.read_csv(self.config.data_path)
        # New Line
        df = pd.read_excel(self.config.data_path)

        # I added this line to see the data path
        logger.info(f' data path: \n{self.config.data_path}')
        logger.info('Read data completed')
        logger.info(f'df dataframe head: \n{df.head().to_string()}')

        ## dropping null values
        df.dropna(inplace = True)

        ## Date of journey column transformation
        df['journey_date'] = pd.to_datetime(df['Date_of_Journey'], format ="%d/%m/%Y").dt.day
        df['journey_month'] = pd.to_datetime(df['Date_of_Journey'], format ="%d/%m/%Y").dt.month

        ## encoding total stops.
        df.replace({'Total_Stops': {'non-stop' : 0, '1 stop': 1, '2 stops': 2, '3 stops': 3, '4 stops': 4}}, inplace = True)

        ## ecoding airline, source, and destination
        df_airline = pd.get_dummies(df['Airline'], dtype=int)
        df_source = pd.get_dummies(df['Source'],  dtype=int)
        df_dest = pd.get_dummies(df['Destination'], dtype=int)

        ## dropping first columns of each categorical variables.
        df_airline.drop('Trujet', axis = 1, inplace = True)
        df_source.drop('Banglore', axis = 1, inplace = True)
        df_dest.drop('Banglore', axis = 1, inplace = True)

        df = pd.concat([df, df_airline, df_source, df_dest], axis = 1)
       
        ## handling duration column
        # df['duration'] = df['Duration'].apply(convert_to_minutes)
        # New Line Added
        df['duration'] = df['Duration'].apply(self.convert_to_minutes)
        upper_time_limit = df.duration.mean() + 1.5 * df.duration.std()
        df['duration'] = df['duration'].clip(upper = upper_time_limit)

        ## encodign duration column
        bins = [0, 120, 360, 1440]  # custom bin intervals for 'Short,' 'Medium,' and 'Long'
        labels = ['Short', 'Medium', 'Long'] # creating labels for encoding

        df['duration'] = pd.cut(df['duration'], bins=bins, labels=labels)
        df.replace({'duration': {'Short':1, 'Medium':2, 'Long': 3}}, inplace = True)
        
        ## dropping the columns
        cols_to_drop = ['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route', 'Dep_Time', 'Arrival_Time', 'Duration', 'Additional_Info', 'Delhi', 'Kolkata']
        df.drop(cols_to_drop, axis = 1, inplace = True)

        logger.info('df data transformation completed')
        logger.info(f' transformed df data head: \n{df.head().to_string()}')
        

        # df.to_csv(self.data_transformation_config.transformed_data_file_path, index = False, header= True)
        # New Line
        df.to_excel(self.config.data_path, index = False, header= True)
        # df.to_excel(self.config.transformed_data_file_path, index = False, header= True)
        # df.to_excel(self.data_transformation_config.transformed_data_file_path, index = False, header= True)
        logger.info("transformed data is stored")
        df.head(1)
        ## splitting the data into training and target data
        X = df.drop('Price', axis = 1)
        y = df['Price']
        
        ## accessing the feature importance.
        select = ExtraTreesRegressor()
        select.fit(X, y)

        # plt.figure(figsize=(12, 8))
        # fig_importances = pd.Series(select.feature_importances_, index=X.columns)
        # fig_importances.nlargest(20).plot(kind='barh')
    
        # ## specify the path to the "visuals" folder using os.path.join
        # visuals_folder = 'visuals'
        # if not os.path.exists(visuals_folder):
        #     os.makedirs(visuals_folder)

        # ## save the plot in the visuals folder
        # plt.savefig(os.path.join(visuals_folder, 'feature_importance_plot.png'))
        # logger.info('feature imp figure saving is successful')

        ## further Splitting the data.
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle = True) 
        logger.info('final splitting the data is successful')
        # logger.info(y_train.shape)
        # logger.info(y_test.shape)        
        

        ## returning splitted data and data_path.
        return (
            X_train, 
            X_test, 
            y_train, 
            y_test,
            self.config.data_path
            # self.data_transformation_config.transformed_data_file_path
        )    

           


# class DataTransformation:
#     def __init__(self, config: DataTransformationConfig):
#         self.config = config

    
#     ## Note: You can add different data transformation techniques such as Scaler, PCA and all
#     #You can perform all kinds of EDA in ML cycle here before passing this data to the model

#     # I am only adding train_test_spliting cz this data is already cleaned up


#     def train_test_spliting(self):
#         data = pd.read_excel(self.config.data_path)

#         # Split the data into training and test sets. (0.75, 0.25) split.
#         train, test = train_test_split(data)

#         train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
#         test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

#         logger.info("Splited data into training and test sets")
#         logger.info(train.shape)
#         logger.info(test.shape)

#         print(train.shape)
#         print(test.shape)
        

In [64]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    # data_transformation.train_test_spliting()
    # New Line
    data_transformation.initiate_data_transformation()
except Exception as e:
    raise e

[2023-11-30 08:55:20,301: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-30 08:55:20,304: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-30 08:55:20,306: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-30 08:55:20,314: INFO: common: created directory at: artifacts]
[2023-11-30 08:55:20,314: INFO: common: created directory at: artifacts/data_transformation]
[2023-11-30 08:55:23,037: INFO: 371139583:  data path: 
artifacts/data_ingestion/flightfare-data.xlsx]
[2023-11-30 08:55:23,037: INFO: 371139583: Read data completed]
[2023-11-30 08:55:23,037: INFO: 371139583: df dataframe head: 
       Airline Date_of_Journey    Source Destination                  Route Dep_Time  Arrival_Time Duration Total_Stops Additional_Info  Price
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL    22:20  01:10 22 Mar   2h 50m    non-stop         No info   3897
1    Air India       1/05/2019   Kolkata    Banglore  CCU →